In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import molsysmt as msm
import numpy as np
from simtk import unit

# Getting distances, neighbor lists and contact maps.

MolSysMT includes a very versatile method to calculate distances between points in space, atoms and/or groups of atoms. As many other methods of this multitool, the method `molsysmt.distance()` has an input argument to choose the engine in charge of getting the result. For instance, `molsysmt.distance()` currently offers two engines `MolSysMT` and `MDTraj`. At this moment only `MolSysMT` will be reported in this guide.

The different options of the method `molsysmt.distance()` will be shown, little by little, along with the following examples.

## The XYZ molecular system form

The first case, and the most simple one, is getting distances from a points distribution in space. MolSysMT accepts a molecular system form where only spatial coordinates are described, with out topological information: the `XYZ` form.

In [4]:
molecular_system = np.zeros([6,3], dtype='float64') * unit.nanometers

In [5]:
msm.get(molecular_system, target='system', form=True)

'XYZ'

The `XYZ` form accepts numpy arrays with length units of the shape $[n\_frames, n\_atoms, 3]$ or $[n\_atoms, 3]$. In case of having an array of rank 2, MolSysMT always understands $n\_frames=1$ and the first rank as the number of atoms:

In [6]:
msm.get(molecular_system, n_frames=True, n_atoms=True)

[1, 6]

Lets create a couple of `XYZ` molecular systems with more than a frame. These two systems will help us illustrate the firts distance calculations:

In [7]:
# Molecular system A with three atoms and three frames.

molecular_system_A = np.zeros([3,3,3], dtype='float64') * unit.nanometers

## First atom
molecular_system_A[0,0,:] = [0, 2, -1] * unit.nanometers
molecular_system_A[1,0,:] = [1, 2, -1] * unit.nanometers
molecular_system_A[2,0,:] = [0, 2, -1] * unit.nanometers

## Second atom
molecular_system_A[0,1,:] = [-1, 1, 1] * unit.nanometers
molecular_system_A[1,1,:] = [-1, 0, 1] * unit.nanometers
molecular_system_A[2,1,:] = [0, 0, 1] * unit.nanometers

## Third atom
molecular_system_A[0,2,:] = [-2, 0, 1] * unit.nanometers
molecular_system_A[1,2,:] = [-2, 0, 0] * unit.nanometers
molecular_system_A[2,2,:] = [-1, 1, 0] * unit.nanometers



# Molecular system B with two atoms and three frames.

molecular_system_B = np.zeros([3,2,3], dtype='float64') * unit.nanometers

## First atom of B
molecular_system_B[0,0,:] = [4, -2, 0] * unit.nanometers
molecular_system_B[1,0,:] = [5, -2, -1] * unit.nanometers
molecular_system_B[2,0,:] = [5, -2, 0] * unit.nanometers

## Second atom of B
molecular_system_B[0,1,:] = [3, 0, -1] * unit.nanometers
molecular_system_B[1,1,:] = [3, 1, 0] * unit.nanometers
molecular_system_B[2,1,:] = [4, 1, 1] * unit.nanometers

## Distance between atoms in space

### Distance between atoms of a system

The first case shows how to get the distance between all points of a system at every frame

In [8]:
distances = msm.distance(molecular_system_A)

The result is an array of rank 3. Where the first axe or rank corresponds to the number of frames and the other two, the second and third axe, accounts for the point or atom indices:

In [9]:
distances.shape

(3, 3, 3)

This way every distance between atoms at each frame is stored. Lets print out the distance between the 0-th and the 2-th atom at frame 1-th:

In [10]:
print('Distance at frame 1-th between atoms 0-th and 2-th: {}'.format(distances[1,0,2]))

Distance at frame 1-th between atoms 0-th and 2-th: 3.7416573867739413 nm


If only the distance between atoms 0-th and 2-th at every frame is required, there is no need to compute $n\_atoms x n\_atoms$ distances. The input arguments `selection_1` and `selection_2` help us to define the range of elements of the output distance matrix:

In [11]:
distances = msm.distance(molecular_system_A, selection_1=0, selection_2=2)

This time the output is an array of rank 3 with shape $[3,1,1]$. The distance for just a pair of atoms was computed for three frames:

In [12]:
distances.shape

(3, 1, 1)

In [13]:
for ii in range(3):
    print('Distance at frame {}-th between atoms 0-th and 2-th: {}'.format(ii,distances[ii,0,0]))

Distance at frame 0-th between atoms 0-th and 2-th: 3.4641016151377544 nm
Distance at frame 1-th between atoms 0-th and 2-th: 3.7416573867739413 nm
Distance at frame 2-th between atoms 0-th and 2-th: 1.7320508075688772 nm


Lets try now to get the distance between the atom 1-th and the atoms 0-th and 2-th at every frame:

In [14]:
distances = msm.distance(molecular_system_A, selection_1=1, selection_2=[0,2])

As you will guess, the output matrix is an array of rank three this time with shape $[3,1,2]$:

In [15]:
distances.shape

(3, 1, 2)

If we want now to print out the distance between atoms 1-th and 2-th for frame 0-th:

In [16]:
print('Distance at frame 0-th between atoms 1-th and 2-th: {}'.format(distances[0,0,1]))

Distance at frame 0-th between atoms 1-th and 2-th: 1.4142135623730951 nm


The position of each atom in lists `selection_1` and `selection_2` is used to locate the corresponding distance in the output array. If instead, you want to use the original atom indices to locate a distance, the input argument `output_form='dict'` can help:

In [17]:
distances = msm.distance(molecular_system_A, selection_1=1, selection_2=[0,2], output_form='dict')

This way the output is no longer a numpy array of rank 3, the output object is now a dictionary of dictionaries of dictionaries. Where the set keys of the first dictionary corresponds to the atom indices of `selection_1`:

In [18]:
distances.keys()

dict_keys([1])

The second nested dictionary has the atom indices of `selection_2` as keys:

In [19]:
distances[1].keys()

dict_keys([0, 2])

And the third and last nested dictionary is defined with the frame indices as keys:

In [20]:
distances[1][2].keys()

dict_keys([0, 1, 2])

Thus, the distance now between atoms 0-th and 2-th in frame 1-th is:

In [21]:
print('Distance at frame 0-th between atoms 1-th and 2-th: {}'.format(distances[1][2][0]))

Distance at frame 0-th between atoms 1-th and 2-th: 1.4142135623730951 nm


Just like `selection_1` and `selection_2` can limit the range of atom indices of the calculation, `frame_indices_1` can be used to define the list of frames where the method applies:

In [22]:
distances = msm.distance(molecular_system_A, selection_1=1, selection_2=[0,2], frame_indices_1=[1,2])

In [23]:
print('Distance at frame 2-th between atoms 1-th and 2-th: {}'.format(distances[1,0,1]))

Distance at frame 2-th between atoms 1-th and 2-th: 1.7320508075688772 nm


You can check again that with `output_form='dict'` the original indics for atoms and frames work to locate a distance:

In [24]:
distances = msm.distance(molecular_system_A, selection_1=1, selection_2=[0,2], frame_indices_1=[1,2], output_form='dict')

In [25]:
print('Distance at frame 2-th between atoms 1-th and 2-th: {}'.format(distances[1][2][2]))

Distance at frame 2-th between atoms 1-th and 2-th: 1.7320508075688772 nm


### Distance between atoms of two systems

The second case shows how to get the distance between atoms of two systems at every frame

In [26]:
distances = msm.distance(item_1=molecular_system_A, item_2=molecular_system_B)

As it was shown previously, the result is an array of rank 3. Again, where the first axe or rank corresponds to the number of frames and the other two, the second and third axe, accounts for the atom indices -this time in each system-:

In [27]:
distances.shape

(3, 3, 2)

Lets print out the distance between atom 1-th of `molecular_system_A` and atom 0-th of `molecular_system_B` at every frame:

In [28]:
for ii in range(3):
    print('Distance at frame {}-th between atom 1-th of A and atom 0-th of B: {}'.format(ii,distances[ii,1,0]))

Distance at frame 0-th between atom 1-th of A and atom 0-th of B: 5.916079783099616 nm
Distance at frame 1-th between atom 1-th of A and atom 0-th of B: 6.6332495807108 nm
Distance at frame 2-th between atom 1-th of A and atom 0-th of B: 5.477225575051661 nm


Now that `item_1` and `item_2` contain different systems, `selection_1` and `selection_2` do not work over the same system as in previous subsection, but over each molecular system (`selection_1` over `item_1` and `selection_2` over `item_2`). Lets get the distance only between atom 1-th of `molecular_system_A` and atom 0-th of `molecular_system_B` for every frame:

In [29]:
distances = msm.distance(item_1=molecular_system_A, selection_1=1, item_2=molecular_system_B, selection_2=0)

In [30]:
distances.shape

(3, 1, 1)

In [31]:
for ii in range(3):
    print('Distance at frame {}-th between atom 1-th of A and atom 0-th of B: {}'.format(ii,distances[ii,0,0]))

Distance at frame 0-th between atom 1-th of A and atom 0-th of B: 5.916079783099616 nm
Distance at frame 1-th between atom 1-th of A and atom 0-th of B: 6.6332495807108 nm
Distance at frame 2-th between atom 1-th of A and atom 0-th of B: 5.477225575051661 nm


Again, the input argument `output_form='dict'` lets us play with the original indices in the output object. As it was described in the previous subsection, this dictionary of dictionaries of dictionaries has three keys: the first one corresponds to the atom indices of `item_1`, the second one corresponds to the atom indices of `item_2` and the third one to the frame indices. This way the distance now between atom 1-th of `molecular_system_A` and atom 0-th of `molecular_system_B` at frame 1-th:

In [32]:
distances = msm.distance(item_1=molecular_system_A, selection_1=1, frame_indices_1=1,
                         item_2=molecular_system_B, selection_2=0, output_form='dict')

In [33]:
print('Distance at frame 1-th between atom 1-th of A and atom 0-th of B: {}'.format(distances[1][0][1]))

Distance at frame 1-th between atom 1-th of A and atom 0-th of B: 6.6332495807108 nm


Notice that `frame_indices_1` was used to define the frame indices where the distance is computed. If only `frame_indices_1` is used, the same list of indices is used sequentially for both systems `item_1` and `item_2`:

In [34]:
distances_1 = msm.distance(item_1=molecular_system_A, selection_1=[1,2], frame_indices_1=[0,2],
                           item_2=molecular_system_B, selection_2=[0,1])

In [35]:
distances_2 = msm.distance(item_1=molecular_system_A, selection_1=[1,2], frame_indices_1=[0,2],
                           item_2=molecular_system_B, selection_2=[0,1], frame_indices_2=[0,2])

In [36]:
distances_1 == distances_2

array([[[ True,  True],
        [ True,  True]],

       [[ True,  True],
        [ True,  True]]])

In [37]:
print('Distance at frame 2-th between atom 2-th of A and atom 1-th of B: {}'.format(distances_1[1,1,1]))

Distance at frame 2-th between atom 2-th of A and atom 1-th of B: 5.0990195135927845 nm


### Distances between atoms positions in different frames

At the end of last subsection we saw how, in addition to the input arguments `selection_1` and `selection_2`, the input arguments `frame_indices_1` and `frame_indices_2` alter the way distances are computed. Lets use the four arguments in the next example to revisit their function:

In [38]:
distances = msm.distance(item_1=molecular_system_A, selection_1=[1,2], frame_indices_1=[0,1],
                         item_2=molecular_system_B, selection_2=1, frame_indices_2=[1,2])

The distance between atoms 1-th and 2-th of `molecular_system_A` and atom 1-th of `molecular_system_B` are computed. But, these distances are not between spatial positions in the same frame index. When two frame indices lists are provided by means of `frame_indices_1` and `frame_indices_2`, pairs of frames are taken sequentially. In this case positions of `item_1` at frame 0-th are confronted againts positions of `item_2` at frame 1-th, and the results are stored in the first frame of the distances output. Then, according to the last cell frame indices, positions of `item_1` at frame 1-th are confronted againts positions of `item_2` at frame 2-th. Thus, lets print out for example the distance between the position of atom 2-th of `item_1` in frame 1-th and the position of atom 1-th of `item_2` in frame 2-th:

In [39]:
print('The distance between atom 2-th of A at frame 1-th and atom 1-th of B at frame 2-th is: {}'.format(distances[1,1,0]))

The distance between atom 2-th of A at frame 1-th and atom 1-th of B at frame 2-th is: 6.164414002968976 nm


In this case, when the output object is a dictionary of dictionaries of dictionaries, the last nested keys correspond to the frame indices in `frame_indices_1`. Lets compute de same distances as before printing out the same specific distance:

In [40]:
distances = msm.distance(item_1=molecular_system_A, selection_1=[1,2], frame_indices_1=[0,1],
                         item_2=molecular_system_B, selection_2=1, frame_indices_2=[1,2], output_form='dict')

In [41]:
print('The distance between atom 2-th of A at frame 1-th and atom 1-th of B at frame 2-th is: {}'.format(distances[2][1][1]))

The distance between atom 2-th of A at frame 1-th and atom 1-th of B at frame 2-th is: 6.164414002968976 nm


The possibility to calculate distances between crossing frame indices opens the door to get displacement lengths as it is shown in next subsection.

### Displacement distances

When both input arguments `frame_indices_1` and `frame_indices_2` are used over a unique set of atoms of the same molecular system, the distances computed acquired a simple physical meaning: displacements. Lets see the following case:

In [43]:
distances = msm.distance(item_1=molecular_system_A, selection_1=1, frame_indices_1=[0,1], frame_indices_2=[1,2])

The shape of the output object is:

In [44]:
distances.shape

(2, 1, 1)

The length of the distance walked by atom 1-th of `molecular_system_A` between frames 0-th and 1-th is:

In [45]:
print('The displacement length of atom 1-th between frames 0-th and 1th is: {}'.format(distances[0,0,0]))

The displacement length of atom 1-th between frames 0-th and 1th is: 1.0 nm


While the displacement length of the same atom between the next two consecutive frames is:

In [46]:
print('The displacement length of atom 1-th between frames 1-th and 2-th is: {}'.format(distances[1,0,0]))

The displacement length of atom 1-th between frames 1-th and 2-th is: 1.0 nm


If we want to get the length distance an atom moves between a fixed frame, 0-th for instance, and the rest of frames we can invoke the command `molsysmt.distance()` this way:

In [47]:
distances = msm.distance(item_1=molecular_system_A, selection_1=1, frame_indices_1=[0,1,2], frame_indices_2=[0,0,0],
                         output_form='dict')

The displacement length of atom 1-th between its position at frame 0-th and all other frames is:

In [48]:
for ii in range(3):
    print('The displacement length of atom 1-th between frames 0-th and {}-th is: {}'.format(ii, distances[1][1][ii]))

The displacement length of atom 1-th between frames 0-th and 0-th is: 0.0 nm
The displacement length of atom 1-th between frames 0-th and 1-th is: 1.0 nm
The displacement length of atom 1-th between frames 0-th and 2-th is: 1.4142135623730951 nm


### Distance between atoms pairs

When the method `molsysmt.distance()` is invoked to calculate the distances between a set of $N$ atoms of `item_1` and a set of $M$ atoms of `item_2`, the result is an object, a numpy array or a dictionary, with $N x M$ distances per frame. Sometimes all these different distances are not needed, but only those between specific atom pairs. These atom pairs can be given by the elements in `selection_1` and `selection_2` coupled sequantially one to one. This option is activated when the input argument `pairs=True`:

In [49]:
distances = msm.distance(item_1=molecular_system_A, selection_1=[0,0,1], selection_2=[1,2,2],
                         frame_indices_1=[1,2], pairs=True)

When `pairs=True` the shape of the output numpy array is $[n\_frames, N]$, where $N$ is the number of pairs: [0,1], [0,2] and [1,2], in this case.  

In [50]:
distances.shape

(2, 3)

Lets print out the distance between atoms 0-th and 2-th in frame 1-th:

In [51]:
print('The distance between the pair of atoms 0-th and 2-th in frame 1-th: {}'.format(distances[0,1]))

The distance between the pair of atoms 0-th and 2-th in frame 1-th: 3.7416573867739413 nm


The dictionary output form also works for atom pairs in the same way as with `pairs=False`:

In [52]:
distances = msm.distance(item_1=molecular_system_A, selection_1=[0,0,1], selection_2=[1,2,2],
                         frame_indices_1=[1,2], pairs=True, output_form='dict')

In [53]:
print('The distance between the pair of atoms 0-th and 2-th in frame 1-th: {}'.format(distances[0][2][1]))

The distance between the pair of atoms 0-th and 2-th in frame 1-th: 3.7416573867739413 nm


### Minimum and Maximum distance

Sometimes the minimum and maximum distance between two sets of atoms needs to be obtained. Although this step could be done with the method `molsysmt.distance()` and  a little coding, MolSysMT includes two methods to make it even easier: `molsysmt.minimum_distance()` and `molsysmt.maximum_distance()`. Lets see in the following cells how they work.

As first example, lets get the minimum distance between both molecular systems A and B:

In [54]:
min_pairs, min_distances = msm.minimum_distance(item_1=molecular_system_A, item_2=molecular_system_B)

The result is offered as two numpy arrays: the list of atoms pairs minimizing the distance for each frame, and the corresponding value of the minimum distance (for each frame also).

In [55]:
min_pairs.shape

(3, 2)

In [56]:
min_pairs

array([[0, 1],
       [0, 1],
       [1, 1]])

In [57]:
min_distances.shape

(3,)

In [58]:
min_distances

Quantity(value=array([3.60555128, 2.44948974, 4.12310563]), unit=nanometer)

In [59]:
print('The minimum distance in frame 2-th is given by atom {}-th of A and atom {}-th of B: {}'.format(min_pairs[2,0], min_pairs[2,1], min_distances[2]))

The minimum distance in frame 2-th is given by atom 1-th of A and atom 1-th of B: 4.123105625617661 nm


All input arguments described in previous subsections can also be used with `molsysmt.minimum_distance()` and `molsysmt.maximum_distance()`. Lets see an example:

In [60]:
min_pairs, min_distances = msm.minimum_distance(item_1=molecular_system_A, selection_1=[0,1,2], selection_2=[0,1,2],
                                               frame_indices_1=[0,1], frame_indices_2=[1,2], pairs=True)

Remember that with `pairs=True`, the output does not longer refer atoms indices but pairs indices. That is the reason why the shape of min_pairs is now:

In [61]:
min_pairs.shape

(2,)

While,

In [62]:
min_distances.shape

(2,)

Which means that the minimum displacement between consecutive frames was observed for:

In [63]:
for ii in range(2):
    print('Atom {}-th had the minimum displacement of A between frames {}-th and {}-th: {}'.format(min_pairs[ii], ii, ii+1, min_distances[ii]))

Atom 0-th had the minimum displacement of A between frames 0-th and 1-th: 1.0 nm
Atom 0-th had the minimum displacement of A between frames 1-th and 2-th: 1.0 nm


There are situations in which we have a list of atoms of a set A and the minimum distance with a second set of atoms B needs to be known for every single atom of A. In this case the first set has to be considered not as entity (as set) in view of getting a single minimum distance. Lets illustrate this with an example:

In [64]:
min_pairs, min_distances = msm.minimum_distance(item_1=molecular_system_A, selection_1=[1,2], frame_indices_1=[0,1,2],
                                                item_2=molecular_system_B, selection_2=[0,1],
                                                as_entity_1=False, as_entity_2=True)

The output corresponds to the minimum distance of atom 1-th of A to any atom of B and the minimum distance of atom 2-th of A to any atom of B, at every frame:

In [65]:
min_pairs.shape

(3, 2)

In [66]:
min_distances.shape

(3, 2)

In [67]:
selection_2=[0,1]
print('Atom 1-th of A has the minimum distance to B with its atom {}-th in frame 1-th: {}'.format(selection_2[min_pairs[1,0]], min_distances[1,0]))

Atom 1-th of A has the minimum distance to B with its atom 1-th in frame 1-th: 4.242640687119285 nm


In [68]:
for ii in range(3):
    print('The {}-th is the closest atom of B to atom 1-th of A at frame {}-th with {}'.format(selection_2[min_pairs[ii,0]],ii, min_distances[ii,0]))

The 1-th is the closest atom of B to atom 1-th of A at frame 0-th with 4.58257569495584 nm
The 1-th is the closest atom of B to atom 1-th of A at frame 1-th with 4.242640687119285 nm
The 1-th is the closest atom of B to atom 1-th of A at frame 2-th with 4.123105625617661 nm


## Neighbor lists

## Contact maps